# A03. Steamer
Sources: 
- steamerprojections.com

In [14]:
# Download historical weekly logs
def download_logs():
    # Weekly log URL
    steamer_url = 'http://www.steamerprojections.com/index.php/projections/historical-weekly-logs'

    # Load in existing web browser
    javascript_code = f"window.open('{steamer_url}', '_blank');"
    display(Javascript(javascript_code))

    # Wait
    time.sleep(5)
    
    # Identify current screen dimensions
    screenWidth, screenHeight = pyautogui.size()

    # Pitcher download link location
    x_coordinate = 500/2560 * screenWidth
    y_coordinate = 800/1600 * screenHeight

    # Download pitchers
    pyautogui.click(x=x_coordinate, y=y_coordinate)   
    
    # Wait
    time.sleep(60)
    
    destination_folder = r'C:\Users\james\Documents\MLB\Database\A03. Steamer'
    
    # Find latest pitcher download
    matching_files = glob.glob(os.path.join(download_path, "steamer_pitchers*"))

    # Move to destination folder
    if matching_files:
        most_recent_file = max(matching_files, key=os.path.getmtime)
        shutil.move(most_recent_file, os.path.join(destination_folder, os.path.basename(most_recent_file)))
        print(f"Moved '{most_recent_file}' to '{destination_folder}'.")
    else:
        print(f"No matching files found in '{download_path}'.")

    
    # Batter download link location
    x_coordinate = 1400/2560 * screenWidth
    y_coordinate = 780/1600 * screenHeight

    # Download hitters
    pyautogui.click(x=x_coordinate, y=y_coordinate)
    
    time.sleep(60)
    
    # Find latest hitter download
    matching_files = glob.glob(os.path.join(download_path, "steamer_hitters*"))

    # Move to destination folder
    if matching_files:
        most_recent_file = max(matching_files, key=os.path.getmtime)
        shutil.move(most_recent_file, os.path.join(destination_folder, os.path.basename(most_recent_file)))
        print(f"Moved '{most_recent_file}' to '{destination_folder}'.")
    else:
        print(f"No matching files found in '{download_path}'.")

# Clean

In [3]:
# %run "U1. Imports.ipynb"
# %run "U2. Utilities.ipynb"
# %run "U3. Classes.ipynb"
# %run "D3. Simulation Functions.ipynb"

# baseball_path = r'C:\Users\james\Documents\MLB\Database'

Code was last run on: 2023-10-26


In [4]:
# db_path = r'C:\Users\james\Documents\MLB\Database\MLBDB.db'
# engine = create_engine(f'sqlite:///{db_path}')

##### 1. Steamer Hitters

In [5]:
# sql_query = f'SELECT * FROM "Steamer Hitters"'
# steamer_hitters_df = pd.read_sql_query(sql_query, con=engine)

In [8]:
# steamer_hitters_df.head()

,proj_year,postseason,proj_date,mlbamid,steamerid,firstname,lastname,Team,position,bats,age,reliability,G,PA,IBB,NIBB,BB,K,HBP,RBOE,SH,SF,AB,H,1B,2B,3B,HR,AVG,OBP,SLG,wOBA,SB,CS,wSB,UBR,BaseRunning,UZR,R,RBI
0,2014,0,2014-06-17,607680.0,12434,Kevin,Pillar,TOR,CF,R,25.0,0.80,22.75,90.8,0.091,3.6094,3.7002,14.6142,0.636,0.8354,0.6198,0.6942,85.1502,22.1426,15.4611,4.8490,0.6143,1.2182,0.260042,0.291613,0.374338,0.298854,2.9539,1.6472,-0.0654,0.109,0.0436,0.169,9.4540,9.0755
1,2014,0,2014-06-17,493128.0,13047,Munenori,Kawasaki,TOR,SS,L,33.0,0.70,0.20,1.0,0.000,0.0825,0.0825,0.1598,0.009,0.0092,0.0071,0.0076,0.8938,0.2090,0.1557,0.0412,0.0076,0.0045,0.233803,0.300456,0.312114,0.283956,0.0187,0.0089,0.0000,0.000,0.0000,0.000,0.0994,0.0776
2,2014,0,2014-06-17,136660.0,1392,Mark,DeRosa,TOR,2b,R,39.0,0.60,0.20,1.0,0.002,0.0914,0.0934,0.2081,0.007,0.0092,0.0051,0.0074,0.8870,0.2052,0.1409,0.0417,0.0036,0.0191,0.231367,0.305661,0.350845,0.300232,0.0070,0.0043,-0.0005,-0.002,-0.0029,0.000,0.1040,0.0965
3,2014,0,2014-06-17,408314.0,1736,Jose,Reyes,TOR,SS,B,31.0,0.86,77.35,358.0,2.506,25.9714,28.4774,35.1186,1.074,3.2936,2.9447,2.3188,323.1850,91.7870,63.0020,19.6804,2.6588,6.4458,0.284007,0.338934,0.421190,0.336366,15.0226,5.5845,0.7520,0.362,1.1136,-2.532,48.9535,34.2312
4,2014,0,2014-06-17,430832.0,1887,Jose,Bautista,TOR,RF,R,33.0,0.87,77.35,346.8,3.815,48.5945,52.4093,54.6331,3.121,3.1906,0.9847,2.6035,287.6812,77.3646,42.7919,15.1100,0.3443,19.1183,0.268925,0.383204,0.523212,0.393728,3.2033,1.7708,-0.1527,0.437,0.2842,-0.951,53.0616,52.8365


In [21]:
def clean_steamer_hitters(df):
    ### Hitting
    # Basic stats
    hit_list = ['1B', '2B', '3B', 'HR', 'BB', 'HBP', 'K']

    # Advance stats
    rate_list = ['OBP', 'SLG', 'wOBA']
    for stat in hit_list:
        rate = stat + "_rate"
        rate_list.append(rate)
        df[rate] = df[stat] / df['PA']

    ### Base running
    # Stolen base attempts
    df['SBA'] = df['SB'] + df['CS']
    # Stolen base opportunities (times on first)
    df['SBO'] = df['1B'] + df['BB'] + df['HBP']
    # Implied stolen base attempt rate
    df['sba_imp'] = df['SBA'] / df['SBO']

    # Cap implied stolen base attempt rat 
    df['sba_imp'] = np.where(df['sba_imp'] > 0.15, 0.15, df['sba_imp'])
    
    # Determine stolen base success rate
    df['sbr'] = df['SB'] / df['SBA']
    
    # Fill in missings
    df['sbr'].fillna(0.6, inplace=True) # assume 25th percentile 
    df['sba_imp'].fillna(0.05, inplace=True) # assume low prob
    
    # Date
    df['date'] = df['proj_date'].str.replace("-", "")
    df['date'] = df['date'].astype('int')
    
    
    # Keep relevant variables
    keep_list = ['date', 'firstname', 'lastname', 'mlbamid', 'steamerid', 'sba_imp', 'sbr'] + rate_list
    df = df[keep_list]
    
    # Clean up
    df.columns = df.columns.str.lower()
    df.rename(columns={'1b_rate': 'b1_rate', '2b_rate': 'b2_rate', '3b_rate': 'b3_rate', 'k_rate':'so_rate'}, inplace=True)
    df.dropna(inplace=True)
    
    # Calculate stolen base attempt and success rates by base
    sba_2b_reg = pickle.load(open(os.path.join(model_path, 'sba_2b_20220901.sav'), 'rb'))
    df['sba_2b'] = sba_2b_reg.predict(df[['sba_imp']])

    sba_3b_reg = pickle.load(open(os.path.join(model_path, 'sba_3b_20220901.sav'), 'rb'))
    df['sba_3b'] = sba_3b_reg.predict(df[['sba_imp']])

    sb_2b_reg = pickle.load(open(os.path.join(model_path, 'sb_2b_20220901.sav'), 'rb'))
    df['sb_2b'] = sb_2b_reg.predict(df[['sbr']])

    sb_3b_reg = pickle.load(open(os.path.join(model_path, 'sb_3b_20220901.sav'), 'rb'))
    df['sb_3b'] = sb_3b_reg.predict(df[['sbr']])
    
    return df 

In [22]:
# steamer_hitters_df2 = clean_steamer_hitters(steamer_hitters_df)

In [23]:
# steamer_hitters_df2.head()

,date,firstname,lastname,mlbamid,steamerid,sba_imp,sbr,obp,slg,woba,b1_rate,b2_rate,b3_rate,hr_rate,bb_rate,hbp_rate,so_rate,sba_2b,sba_3b,sb_2b,sb_3b
0,20140617,Kevin,Pillar,607680.0,12434,0.15000,0.641999,0.291613,0.374338,0.298854,0.170276,0.053403,0.006765,0.013416,0.040751,0.007004,0.160949,0.091844,0.029306,0.681214,0.617688
1,20140617,Munenori,Kawasaki,493128.0,13047,0.11165,0.677536,0.300456,0.312114,0.283956,0.155700,0.041200,0.007600,0.004500,0.082500,0.009000,0.159800,0.068762,0.021021,0.705031,0.626325
2,20140617,Mark,DeRosa,136660.0,1392,0.04683,0.619469,0.305661,0.350845,0.300232,0.140900,0.041700,0.003600,0.019100,0.093400,0.007000,0.208100,0.029749,0.007015,0.666114,0.612211
3,20140617,Jose,Reyes,408314.0,1736,0.15000,0.729001,0.338934,0.421190,0.336366,0.175983,0.054973,0.007427,0.018005,0.079546,0.003000,0.098097,0.091844,0.029306,0.739522,0.638835
4,20140617,Jose,Bautista,430832.0,1887,0.05059,0.643996,0.383204,0.523212,0.393728,0.123391,0.043570,0.000993,0.055128,0.151123,0.008999,0.157535,0.032012,0.007828,0.682552,0.618173


##### 2. Steamer Pitchers

In [24]:
# sql_query = f'SELECT * FROM "Steamer Pitchers"'
# steamer_pitchers_df = pd.read_sql_query(sql_query, con=engine)

In [25]:
# steamer_pitchers_df.head()

,proj_year,postseason,proj_date,mlbamid,steamerid,firstname,lastname,IP,G,GS,start_percent,start_IP,relief_IP,SV,League,DBTeamId,FBV,reliability,Throws,age,Krate,BBrate,HBPrate,GBrate,FBrate,LDrate,SIERA,BABIP,TBF,batters_per_IP,ERA,RA,K,BB,HR,HBP,SH,GB,FB,LD,K9,BB9,HR9,H,ER,R,WHIP,W,L
0,2014,0,2014-06-17,518886.0,6655,Craig,Kimbrel,37.314815,37.314815,0.0,0.0,0.0,37.314815,19.403704,NL,ATL,96.7,0.58064,R,26,0.3867,0.0784,0.009198,0.455,0.335,0.210571,1.92,0.287870,147.0,3.93,1.6703,1.9213,56.7426,11.5041,1.98,1.3497,1.5407,34.3973,25.3255,15.9189,13.6858,2.7747,0.4786,23.1755,6.9251,7.9658,0.92938,3.0390,0.6925
1,2014,0,2014-06-17,547973.0,10233,Aroldis,Chapman,37.716049,37.716049,0.0,0.0,0.0,37.716049,18.974074,NL,CIN,100.3,0.55961,L,26,0.4227,0.0971,0.009827,0.410,0.388,0.202224,1.85,0.276936,149.0,3.95,1.6840,1.9115,63.0390,14.4809,2.74,1.4655,1.5659,28.1190,26.6101,13.8691,15.0427,3.4555,0.6532,20.9723,7.0573,8.0104,0.94000,3.0659,0.7057
2,2014,0,2014-06-17,493157.0,9227,Koji,Uehara,36.913580,36.913580,0.0,0.0,0.0,36.913580,17.434568,AL,BOS,88.0,0.55498,R,39,0.3079,0.0432,0.005769,0.355,0.433,0.212860,2.34,0.278315,145.0,3.93,1.9319,2.1613,44.6414,6.2634,3.70,0.8364,0.7829,32.8242,40.0362,19.6815,10.8841,1.5271,0.9029,28.4064,7.9239,8.8647,0.93922,2.8990,0.7924
3,2014,0,2014-06-17,445276.0,3096,Kenley,Jansen,36.111111,36.111111,0.0,0.0,0.0,36.111111,17.888889,NL,LAN,94.1,0.59125,R,26,0.3474,0.0759,0.008468,0.391,0.409,0.199716,2.27,0.278779,144.0,3.99,2.0793,2.3203,50.0798,10.9415,2.93,1.2207,1.5136,31.4366,32.8838,16.0573,12.4814,2.7269,0.7299,24.5260,8.3429,9.3099,0.98218,2.7768,0.8343
4,2014,0,2014-06-17,518813.0,7196,Greg,Holland,37.314815,37.314815,0.0,0.0,0.0,37.314815,19.001852,AL,KCA,96.0,0.58857,R,28,0.3333,0.0790,0.005115,0.437,0.358,0.205506,2.41,0.287236,149.0,3.99,2.2418,2.5109,49.5894,11.7539,2.48,0.7610,0.8034,37.5276,30.7434,17.6479,11.9605,2.8349,0.5989,26.4365,9.2946,10.4104,1.02346,2.8020,0.9295


In [30]:
def clean_steamer_pitchers(df):
    # Hits per 9 innings
    df['H9'] = df['H'] / df['IP'] * 9
    
    # Calculate average innings per game started
    df['IP_start'] = df['start_IP'] / df['GS']
    df['IP_start'].fillna(0, inplace=True)
    # Replace infinites
    df['IP_start'].replace([np.inf, -np.inf], 3, inplace=True)

    
    # Date
    df['date'] = df['proj_date'].str.replace("-", "")
    df['date'] = df['date'].astype('int')
    
    # Keep relevant variables
    keep_list = ['date', 'firstname', 'lastname', 'mlbamid', 'steamerid'] + pitcher_stats_fg2 
    df = df[keep_list]
    
    return df

In [31]:
# steamer_pitchers_df2 = clean_steamer_pitchers(steamer_pitchers_df)

In [32]:
# steamer_pitchers_df2.head()

,date,firstname,lastname,mlbamid,steamerid,H9,HR9,K9,BB9
0,20140617,Craig,Kimbrel,518886.0,6655,5.589724,0.4786,13.6858,2.7747
1,20140617,Aroldis,Chapman,547973.0,10233,5.004519,0.6532,15.0427,3.4555
2,20140617,Koji,Uehara,493157.0,9227,6.925841,0.9029,10.8841,1.5271
3,20140617,Kenley,Jansen,445276.0,3096,6.112634,0.7299,12.4814,2.7269
4,20140617,Greg,Holland,518813.0,7196,6.376248,0.5989,11.9605,2.8349
